In [1]:
from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import numpy as np
import re

import pdfplumber
import openpyxl

from Script import ExtractPDFTables


# Table Extraction and Mapping (SGD-GRI Link)

In [ ]:
df = read_pdf('ESG-Frameworks/Mapping-Standards/SDG-GRI/sdg-gri.pdf', stream=True, pages = 3, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )

for i in range(4, 73):
    df[0] = df[0].append(read_pdf('ESG-Frameworks/Mapping-Standards/SDG-GRI/sdg-gri.pdf', stream=True, pages = i, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )[0], ignore_index=True)

df[0].rename(columns = {'Sources':'Source'}, inplace = True)

for i in range(74, 99):
    df[0] = df[0].append(read_pdf('ESG-Frameworks/Mapping-Standards/SDG-GRI/sdg-gri.pdf', stream=True, pages = i, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )[0], ignore_index=True)
    
df[0].to_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI/SDG-GRI-DF.csv')


## Structuring Dataframe SDG-GRI

In [11]:
df = pd.read_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI/SDG-GRI-DF.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df = df.dropna()
df = df.drop(['Available Business Disclosures'], axis=1)
df = df.drop(labels = 'Target',axis = 1).groupby(df['Target'].mask(df['Target']==' ').ffill()).agg(', '.join).reset_index()
df.to_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI/SDG-GRI.csv')
df.head(3)

,Target,Disclosure,Source
0,1.1,"207-1, 207-2, 207-3, 207-4","GRI 207: Tax 2019, GRI 207: Tax 2019, GRI 207:..."
1,1.2,"202-1, 203-2","GRI 202: Market Presence, GRI 203: Indirect"
2,1.3,"207-1, 207-2, 207-3, 207-4","GRI 207: Tax 2019, GRI 207: Tax 2019, GRI 207:..."


## Solving Disclousure Column extraction Issue 

In [ ]:
# df['Target_'] = df['Target']

In [ ]:
# df = df[["Target", "Target_", "Available Business Disclosures", "Disclosure"]]
df = df[["Target", "Available Business Disclosures", "Disclosure"]]
# df

In [ ]:
df = df.drop(labels = 'Target',axis = 1).groupby(df['Target'].mask(df['Target']=='').ffill()).agg(' '.join).reset_index()



In [ ]:
# df.rename(columns = {'Target_':'Target'}, inplace = True)

In [ ]:
df.to_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI#2.csv')


## Structuring Dataframe GRI-SDG

In [12]:
df = pd.read_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI/SDG-GRI-DF.csv')
df = df.drop(['Unnamed: 0'], axis=1)
# df = df.replace(np.nan, '', regex=True)
df = df.drop(['Available Business Disclosures'], axis=1)
df = df.dropna()
df.to_csv('ESG-Frameworks/Mapping-Standards/SDG-GRI/GRI-SDG.csv')
df.head(3)


,Target,Disclosure,Source
0,1.1,207-1,GRI 207: Tax 2019
7,1.1,207-2,GRI 207: Tax 2019
17,1.1,207-3,GRI 207: Tax 2019


## Mapping Collected Dataframe (SDG-GRI)

In [ ]:
pdf_tables = 'ESG-Frameworks/Mapping-Standards/SDG-GRI/SDG-GRI.csv'
excelFile = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "SDG's"
regex = '[+-]?[0-9]+\.-?[0-9a-zA-Z_]+'

wb = openpyxl.load_workbook(excelFile)

ws = wb[sheet]
# ws = wb.active

rows = ws.max_row

pdf_tables = pd.read_csv(pdf_tables)

for i in range(1, rows):
    
    if ws.cell(row=i, column=1).value == None:
        pass
    
    else:
        target_cell = ws.cell(row=i, column=1).value
        if(re.search(regex, target_cell)):
            target = re.search(regex, target_cell).group()
            
            try:
                value_to_add = pdf_tables.loc[pdf_tables['Target'] == target]['Disclosure'].item()
                ws.cell(row=i, column=3, value=str(value_to_add))
            except:
                pass

           
wb.save("ESG-Frameworks/Outputs/testing.xlsx")
            


        

# wb.save("ESG-Frameworks/Outputs/testing_2.xlsx")

# ws.cell(row=5, column=1).value

## Mapping Collected Dataframe (GRI-SDG)

In [ ]:
pdf_tables = 'ESG-Frameworks/Mapping-Standards/SDG-GRI/GRI-SDG.csv'
excelFile = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet_2 = "GRI 2016"

wb = openpyxl.load_workbook(excelFile)

ws = wb[sheet_2]


rows = ws.max_row

pdf_tables = pd.read_csv(pdf_tables)

for i in range(1, rows):
    
    if ws.cell(row=i, column=2).value != None:
        
        target_cell = ws.cell(row=i+1, column=2).value
        if target_cell != None:
            try:
                
                if len(pdf_tables[pdf_tables['Disclosure'] == target_cell]) != 0:
                    value_to_add = pdf_tables[pdf_tables.Disclosure==target_cell].squeeze()['Target'].values
                    ws.cell(row=i+1, column=4, value=', '.join(value_to_add))
            except:
                pass
        


           
wb.save("ESG-Frameworks/Outputs/testing.xlsx")
            


# Table Extraction and Mapping (COH4B-GRI Link)

In [ ]:
pdf = pdfplumber.open('ESG-Frameworks/Mapping-Standards/GRI-COH4B/gri-coh4b.pdf')

frames = []

for i in range(12, len(pdf.pages)):
    
    try:
        page = pdf.pages[i]
        table = page.extract_table()
        frames.append(pd.DataFrame(table))
    except:
        pass

df =  pd.concat(frames)
df = df.drop_duplicates()
headers = df.iloc[0]
df = pd.DataFrame(df.values[1:], columns=headers)
df.rename(columns = {'A. COHBP & \ndefinition':'id', None:'A. COHBP & \ndefinition'}, inplace = True)
df['id'] = df['id'].str[:-1] + df['id'].str[-1] + '.'


values = []
for i in range(0, len(df['D. GRI disclosures'])):
    
    match = re.search(r'[0-9]{3}-[0-9]{2}|[0-9]{3}-[0-9]{1}', df['D. GRI disclosures'][i])
    if i < rows:
        try:
            # print(df['D. GRI disclosures'][i][match.start():match.end()])
            values.append(df['D. GRI disclosures'][i][match.start():match.end()])
        
        except:
            values.append('No value :(')
            pass

df['GRI Standards'] = values




df.to_csv('ESG-Frameworks/Mapping-Standards/GRI-COH4B/GRI-COH4B.csv')

## Mapping Collected Dataframe (COH4B-GRI)

In [ ]:
pdf_tables = 'ESG-Frameworks/Mapping-Standards/GRI-COH4B/GRI-COH4B.csv'
excelFile = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "COH4B"
regex = "[+-]?[0-9]+\."

wb = openpyxl.load_workbook(excelFile)

ws = wb[sheet]


rows = ws.max_row

pdf_tables = pd.read_csv(pdf_tables)
pdf_tables['id'] = pdf_tables['id'].astype(str).apply(lambda x: x.replace('.0','.'))

for i in range(1, rows):
    
    if ws.cell(row=i, column=1).value == None:
        pass
    
    else:
        target_cell = ws.cell(row=i, column=1).value
        # print(target_cell)
        if(re.search(regex, target_cell)):
            target = re.search(regex, target_cell).group() 
            # print(target)
            try:
                value_to_add = [pdf_tables.loc[pdf_tables['id'] == target]['C. GRI \nStandards'].item(),
                                pdf_tables.loc[pdf_tables['id'] == target]['D. GRI disclosures'].item()]
                # print(value_to_add)
                ws.cell(row=i, column=3, value=str(value_to_add[0]))
                ws.cell(row=i, column=4, value=str(value_to_add[1]))
                
            except:
                pass

           
wb.save("ESG-Frameworks/Outputs/testing.xlsx")
            

## Mapping Collected Dataframe (GRI-COHB4)

In [ ]:
pdf_tables = 'ESG-Frameworks/Mapping-Standards/GRI-COH4B/GRI-COH4B.csv'
excelFile = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"
# regex = "[+-]?[0-9]+\."
regex = '^[0-9\.\-\/]+$'

wb = openpyxl.load_workbook(excelFile)

ws = wb[sheet]

rows = ws.max_row

pdf_tables = pd.read_csv(pdf_tables)
pdf_tables['id'] = pdf_tables['id'].astype(str).apply(lambda x: x.replace('.0','.'))


for i in range(1, rows):
    if ws.cell(row=i, column=2).value == None:
        # print(i)
        pass
    
    else:
        target_cell = ws.cell(row=i, column=2).value
        if(re.search(regex, target_cell)):
            target = re.search(regex, target_cell).group() 
            
            try:
                value_to_add = pdf_tables.loc[pdf_tables['GRI Standards'] == target]['id'].item()
                ws.cell(row=i, column=6, value=str(value_to_add[0]))
               
  
            except:
                
                pass
wb.save("ESG-Frameworks/Outputs/testing.xlsx")
            



# Table Extraction and Mapping (TCFD-GRI Link)

In [13]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/TCFD-GRI/TCFD to GRI.pdf'
firstTablePage = 1

TCFD_GRI = ExtractPDFTables(pdf_path, firstTablePage)

In [28]:
df = TCFD_GRI.getTables1()

In [20]:
df.to_csv('TCFD-GTI-OUTOUT-2.csv')

In [26]:
newHeader = 2

df = TCFD_GRI.setHeaders(df, newHeader)

In [30]:
df.head(20)

,0,1,2,3,4
0,GRI Standards,None,None,None,None
1,,None,None,None,None
2,Recommended,Related,Description,Omissions and/or content difference(s),Level of
3,Disclosures,code/,,,alignment
4,(TCFD Framework),paragraph,None,None,
5,,,None,None,None
6,Governance,GRI 102-20,Disclosure 102-20 – Executive-level responsibi...,Omissions,Reasonable
7,b. Describe,,a. Whether the organisation has appointed an e...,None.,
8,management’s,,"economic, environmental, and social topics.",Content difference(s),
9,role in assessing,,b. Whether post holders report directly to the...,GRI Standards require reporting organisations ...,
